In [16]:
import numpy as np
import pygame
pygame.init()
pygame.font.init()

def makemaze():
    A = [[],[],[],[],[]]
    for i in range(5):
        for j in range(5):
            A[i].append((np.random.randint(2),np.random.randint(2)))
        A[i].append((1,0))
    A.append([(0,1),(0,1),(0,1),(0,1),(0,1)])
    for i in range(5):
        A[i][0] = (1, np.random.randint(2))
        A[0][i] = (np.random.randint(2), 1)
    A[5].append((0,0))
    return A

def binary(q):
    k = 0
    for i in range(len(q)):
        k += q[len(q) - 1 - i]*(2**i)
    return k

def move(A, i, j):
    B = A[i][j] 
    q = list(A[i+1][j])[1]
    if q == 0:
        B = list(B)
        B.append(0)
    else:
        B = list(B)
        B.append(1)
    p = list(A[i][j+1])[0]
    if p == 0:
        B = list(B)
        B.append(0)
    else:
        B = list(B)
        B.append(1)
    return B

def reward(P, R):
    T = [[],[],[],[],[]]
    for i in range(5):
        for j in range(5):
            m = 0
            z = P[i][j].count(0)
            if P[i][j][0] == 0:
                m = m + R[i-1][j]
            if P[i][j][1] == 0:
                m = m + R[i][j-1]
            if P[i][j][3] == 0:
                m = m + R[i+1][j]
            if P[i][j][2] == 0:
                m = m + R[i][j+1]
            if z != 0:
                m = m/z
            T[i].append(m)
            T[i][j] = T[i][j] - 1
    T[0][0] = 0
    return T

def policy(Q, R):
    P = [[],[],[],[],[]]
    for i in range(5):
        for j in range(5):
            E = []
            if i != 0:
                E.append(R[i-1][j])
            else:
                E.append('n')
            if j != 0:
                E.append(R[i][j-1])
            else:
                E.append('n')
            if j != 4:
                E.append(R[i][j+1])
            else:
                E.append('n')
            if i != 4:
                E.append(R[i+1][j])
            else:
                E.append('n')
            P[i].append(E)
    for i in range(5):
        for j in range(5):
            for k in range(4):
                if Q[i][j][k] == 1:
                    P[i][j][k] = 'n'
    for i in range(5):
        for j in range(5):
            l = [x for x in P[i][j] if x != 'n']
            if l != []:
                q = max(l)
            else:
                q = 1
            for k in range(4):
                if P[i][j][k] == q:
                    P[i][j][k] = 0
                else:
                    P[i][j][k] = 1
    return P

def iterate(Q,t):
    R = [[0,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1]]
    l = 0
    while True:
        P = policy(Q,R)
        R = reward(P,R)
        l= l + 1
        if l >= t:
            break
    return (P, R)

def display(A):    
    ARROWS = [pygame.image.load('rdul32x32.png'), pygame.image.load('url32x32.png'), pygame.image.load('uld32x32.png'), pygame.image.load('ul32x32.png'), pygame.image.load('urd32x32.png'), pygame.image.load('ur32x32.png'), pygame.image.load('ud32x32.png'), pygame.image.load('u32x32.png'), pygame.image.load('rld32x32.png'), pygame.image.load('rl32x32.png'), pygame.image.load('ld32x32.png'), pygame.image.load('l32x32.png'), pygame.image.load('rd32x32.png'), pygame.image.load('r32x32.png'), pygame.image.load('d32x32.png')]

    win = pygame.display.set_mode((1000,500))
    pygame.display.set_caption("Maze")
    clock = pygame.time.Clock()

    class tile(object):
        def __init__(self,x,y,a,b):
            self.x = x
            self.y = y
            self.a = a
            self.b = b

        def draw(self,win):
            if self.a == 1 and self.b == 1:
                pygame.draw.rect(win, (255,255,255), (self.x, self.y, 10, 50))
                pygame.draw.rect(win, (255,255,255), (self.x, self.y, 50, 10))
            if self.a == 1 and self.b == 0:
                pygame.draw.rect(win, (255,255,255), (self.x, self.y, 50, 10))
            if self.a == 0 and self.b == 1:
                pygame.draw.rect(win, (255,255,255), (self.x, self.y, 10, 50))

    font = pygame.font.Font(None, 25)
                
    def redrawgamewindow():
        win.fill((0,0,0))
        for i in range(36):
                box[i].draw(win)
        pygame.draw.rect(win,(0,255,0), (50,50,50,50))
        for i in range(36):
                box1[i].draw(win)
        pygame.draw.rect(win,(0,255,0), (400,50,50,50))
        for i in range(5):
            for j in range(5):
                if (i,j) != (0,0):
                    q = move(A,i,j)
                    k = binary(P[j][i])
                    if k != 15:
                        win.blit(ARROWS[k], (415 + 50*i, 65+ 50*j))
        it = str(iterations)
        string  = "ITERATIONS: " + it
        text = font.render(string, True, (255,255,255))
        win.blit(text, (400,400))
 
    box = []
    box1 = []
    for i in range(6):
            for j in range(6):
                a = A[i][j][0]
                b = A[i][j][1]
                box.append(tile(50 + 50*i, 50 + 50*j, a, b))
    for i in range(6):
            for j in range(6):
                a = A[i][j][0]
                b = A[i][j][1]
                box1.append(tile(400 + 50*i, 50 + 50*j, a, b))
    
    run = True
    R = [[0,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1]]
    Q = [[],[],[],[],[]]
    P =  [[],[],[],[],[]]
    f = 0
    for i in range(5):
        for j in range(5):
            Q[i].append(move(A,j,i))
            P[i].append(move(A,j,i))
    
    iterations = 0
    
    while run:
        clock.tick(2)
        
        redrawgamewindow()
        pygame.display.update()
        
        iterations += 1
        
        for event in pygame.event.get():
            if event == pygame.quit:
                pygame.quit()
                run = False
                
        R = reward(P,R)
        T = policy(Q,R)
        if P == T:
            f = 1
        else:
            P = T
        
        if f == 1:
            J = reward(P,R)
            K = reward(P,J)
            L = reward(P,K)
            for i in range(5):
                for j in range(5):
                    if K[i][j] != L[i][j]:
                        pygame.draw.rect(win, (255,0,0), (410 + 50*j, 60 + 50*i , 40, 40))
                        pygame.display.update()
                    if Q[i][j] == [1,1,1,1]:
                        pygame.draw.rect(win, (255,0,0), (410 + 50*j, 60 + 50*i , 40, 40))
                        pygame.display.update()
            pygame.draw.rect(win,(0,255,0), (400,50,50,50))
            pygame.display.update()
            run = False
    
    for event in pygame.event.get():
            if event == pygame.quit:
                pygame.quit()
                run = False

In [18]:
A = makemaze()
display(A)